# 电商平台零售数据分析


## 一、提出问题

* 项目背景
* 此次项目数据来自Kaggle，包含了2010年12月1日至2011年12月9日在英国注册的非实体网上零售发生的所有交易。公司主要销售独特的全天候礼品。该公司的许多客户都是批发商。

分析目的
* 用户分类（RFM模型），对比分析不同用户群体在时间、地区等维度下交易量，交易金额指标，并根据分析结果提出优化建议。

## 二. 理解数据


In [ ]:
# 忽略警告提示
import warnings
warnings.filterwarnings('ignore')
#导入处理数据包
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
os.chdir(r'E:\云开明培训机构\云开见明培训课件\data summary\第六章')

In [ ]:
#导入可视化库
import plotly as py 
import plotly.graph_objs as go
py.offline.init_notebook_mode()# 初始化步骤
pyplot = py.offline.iplot  #画图函数

In [ ]:
# 读取数据
df  = pd.read_csv('data.csv',encoding='utf-8',dtype= {'CustomerID':str})

In [ ]:
df.shape

In [ ]:
df.info()

* 字段解释：

InvoiceNo --> 订单号码： 6位字符串

StockCode --> 产品代码： 6位字符串

Description --> 产品描述

Quantity --> 产品数量：交易产品数量

InvoiceDate --> 订单日期：订单发生的日期和时间

UnitPrice --> 单价：浮点数值

CustomerID --> 顾客ID：5位字符串

Country --> 国家：客户所在地


## 三.  数据清洗

In [ ]:
#统计缺失率
df.apply(lambda x: sum(x.isnull())/len(x), axis=0)

In [ ]:
# Description字段对于分析结果影响不大，选择直接删除改变量
df.drop(['Description'],axis=1,inplace=True)
# CustomerID字段为客户编号，不能直接删除，选择填充为U（表示未知）
df['CustomerID'] = df['CustomerID'].fillna('U')

* 为方便后续分析，将订单日期拆分，增加年月日时间四个字段；增加合计购买字段；

In [ ]:
#增加合计购买金额（amount）=数量*金额
df['amount']= df['Quantity']*df['UnitPrice']
#订单日期切分
df['data']=[x.split(' ')[0] for x in df['InvoiceDate']] 
df['time']=[x.split(' ')[1] for x in df['InvoiceDate']]
df.drop(['InvoiceDate'],axis=1,inplace=True)
df['year']=[x.split('/')[2] for x in df['data']]
df['month']=[x.split('/')[0] for x in df['data']]
df['day']=[x.split('/')[1] for x in df['data']]
#转换日期格式
df['date'] = pd.to_datetime(df['data'])
df.head()

In [ ]:
# 重复值处理
df = df.drop_duplicates()

In [ ]:
# 异常值处理
df.describe()

In [ ]:
# 查看数据发现商品数量和单价存在负值，进一步查看异常数据。
# 商品数量异常
df1 = df.loc[df['Quantity'] <= 0]
print('异常数据比例：',df1.shape[0]/df.shape[0])

* 从输出结果可以看出，数量为负数的订单都是订单号为‘C’开头的退货订单（实际业务中应找相关部门确认数据异常原因），先把这部分数据存放在另一个数据框内，后续进行分析。

In [ ]:
df2 = df.loc[df['UnitPrice'] <= 0]
print('异常数据比例：',df2.shape[0]/df.shape[0])

In [ ]:
df2['UnitPrice'].head(10)

In [ ]:
#查看数据发现单价为0的居多，查看单价分类
df2['UnitPrice'].groupby(df2['UnitPrice']).count()

* 查询单价异常结果一共2512条，其中2510条为单价为0，预计为促销赠品；还有2条单价是负数，明细为坏账记录。本次不做分析。

## 四、数据分析

### 1、退货订单分析

* 指标一：退货率=退货合计金额/合计金额

In [ ]:
# 退货合计金额
tt = pd.pivot_table(df1,index=['year'],columns=['month'], values=['amount'], aggfunc={'amount':np.sum}, margins=False)
sale_no = tt.iloc[0:2]
sale_no.fillna(0)

In [ ]:
# 合计金额
df2 = df[(df['Quantity'] > 0)&(df['UnitPrice'] > 0)]
pp = pd.pivot_table(df2,index=['year'],columns=['month'], values=['amount'], aggfunc={'amount':np.sum}, margins=False)
sale = pp.iloc[0:2]
sale.fillna(0)

In [ ]:
# 退货率
rate = round(tt/pp)

In [ ]:
rate

* 可以看到，退货率最高为2011年1月和12月，将结果可视化

* 2011年1月和12月退货率均高于平均退货率9%。特别是2011年12月，具体情况应该与相关部门沟通是什么原因导致，并且分析前几年的退货率，是否存在同样的问题。

### 2、用户分类

RFM模型是衡量客户价值和客户创利能力的重要工具和手段。通过一个客户的近期购买行为、购买的总体频率以及花了多少钱三项指标来描述该客户的价值状况。
* R：最近一次消费时间（最近一次消费到参考时间的间隔）
* F：消费的频次(消费了多少次）
* M：消费的金额 （总消费金额）

In [ ]:
#客户最近一次消费时间
R_value=df2.groupby('CustomerID')['date'].max()
#计算客户最后一次消费距离截止日期的天数
R = (df2['date'].max()- R_value).dt.days

#客户消费频率
F = df2.groupby('CustomerID')['InvoiceNo'].nunique()

#客户消费金额
M = df2.groupby('CustomerID')['amount'].sum()

* 分别查看R/F/M值，对其设置合理阈值，进行分层

In [ ]:
R.describe()

In [ ]:
import seaborn as sns
sns.set(style='darkgrid',context='notebook',font_scale=1.2) # 设置背景
# 支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.hist(R,bins =30)
plt.show()

* 可以看到，截止2011年12月9日，最后一次消费距离截止日期的天数平均为92天，方差为100，波动较大。距离天数最大超过一年。

In [ ]:
# 查看F值
F.describe()

In [ ]:
plt.hist(F,bins =30)
plt.show

* 由于极值的影响，可视化不明显。

In [ ]:
# 查看频次小于50的
plt.hist(F[F<50],bins =30)
plt.show

* 可知，在2010年12月1日至2011年12月9日时间内，用户平均消费4.6次。其中，有用户在此期间消费了1428次，前面理解数据可知，该公司有许多客户都是批发商。所以不存在异常。

In [ ]:
# 查看M值
M.describe()

In [ ]:
plt.hist(M,bins =30)
plt.show

In [ ]:
# 查看金额小于2000的分布

In [ ]:
plt.hist(M[M<2000],bins =100)
plt.show

* 可知，因存在极值，平均值会被影响，通过查看中位数可知整体平均金额为668元。

### 针对之前的分析，对用户进行分级

* 分别计算r,f,m得分

In [ ]:
R_bins  = [0,30,90,180,360,720]
F_bins = [1,2,5,10,20,500]
M_bins = [0,500,2000,5000,10000,200000]

In [ ]:
# 数据离散化
R_score = pd.cut(R,R_bins,labels = [5,4,3,2,1],right=False)
F_score = pd.cut(F,F_bins,labels = [1,2,3,4,5],right=False)
M_score = pd.cut(M,M_bins,labels = [1,2,3,4,5],right=False)

In [ ]:
rfm = pd.concat([R_score,F_score,M_score],axis=1) #数据合并

In [ ]:
rfm.rename(columns={'date':'R_score','InvoiceNo':'F_score','amount':'M_score'},inplace = True)

In [ ]:
rfm.head()

In [ ]:
#数据类型转换
for i in ['R_score','F_score','M_score']:
    rfm[i] = rfm[i].astype(float)   

In [ ]:
rfm.describe()

* 这里使用R/F/M得分的平均值为阈值，得分大于平均值设置为分值高，小于分值设置为分值低

In [ ]:
rfm['R'] = np.where(rfm['R_score']>3.82,'高','低')

In [ ]:
rfm['F'] = np.where(rfm['F_score']>2.03,'高','低')

In [ ]:
rfm['M'] = np.where(rfm['M_score']>1.89,'高','低')

In [ ]:
rfm.head()

In [ ]:
rfm['value'] = rfm['R'].str[:] + rfm['F'].str[:]+rfm['M'].str[:] #字符串拼接

In [ ]:
rfm['value'] = rfm['value'].str.strip()#去除字符串空格

In [ ]:
def trans_value(x):
    if x =='高高高':
        return '重要价值客户'
    elif x =='高低高':
        return '重要发展客户'
    elif x =='低高高':
        return '重要保持客户'
    elif x =='低低高':
        return '重要挽留客户'
    elif x =='高高低':
        return '一般价值客户'
    elif x =='高低低':
        return '一般发展客户'
    elif x =='低高低':
        return '一般保持客户'
    else:
        return '一般挽留客户'

In [ ]:
rfm['用户等级'] = rfm['value'].apply( trans_value)

In [ ]:
rfm['用户等级'].value_counts()

In [ ]:
# 对结果可视化
trace_basic = [go.Bar(x =rfm['用户等级'].value_counts().index.tolist(),y =rfm['用户等级'].value_counts().values.tolist(),
                      marker=dict(color='orange'),opacity=0.50)] #透明度
layout = go.Layout(title = '用户等级情况', xaxis =dict(title ='用户重要度'))
figure_basic = go.Figure(data = trace_basic,layout=layout)# data与layout组成一个图象对象
pyplot(figure_basic) #输出

In [ ]:
trace = [go.Pie(labels = rfm['用户等级'].value_counts().index, values =rfm['用户等级'].value_counts().values,hole=0.2,  \
               textfont =dict(size=12,color ='white'))]                 # hole中间部分
layout = go.Layout(title = '用户等级比例')
fig = go.Figure(data = trace,layout=layout) # data与layout组成一个图象对象
pyplot(fig) #输出

五 结论和建议


* 1、针对退货订单，2011年1月与11月退货率存在异常，需要了解是外部因素还是内部因素导致的。从产品，渠道，价格，促销四个方面分析原因，找到原因，进行改进。

* 2、用户分层，得到8个类别的客户。根据数据可知，该公司用户数最多的为重要价值客户，重要发展客户，占总用户数的47%。对于重要发展客户，由于最近交易较少，建议及时推送公司活动及相关产品信息，唤回客户；其次最多的的为一般发展用户和一般挽留客户，占总用户数40%。针对一般发展用户，获取该类用户详细数据进行用户画像，了解用户消费需求，及时推送产品信息；对于重要保持客户和重要挽留客户，以赠送优惠券或者推送折扣信息等措施增加用户活跃度。
* 在真实项目中，由于给客户进行营销会有相关成本，应该按照客户的特征和比例进行相应的分配,已使得营销费用利用率最大化